In [10]:
import os
import pickle

import numpy as np

In [2]:
cma_es_path = 'bilevel_opt_result/cma_es/3_5.pkl'
cma_es = pickle.load(open(cma_es_path, 'rb'))
print(cma_es.keys())
optimal_cma_es_path = 'bilevel_opt_result/optimal/cma_es/optimal_result_3_5.pkl'
optimal_cma_es = pickle.load(open(optimal_cma_es_path, 'rb'))
print(optimal_cma_es.keys())

dict_keys(['opt_action_pos', 'opt_log'])
dict_keys(['x_trajectory_list', 'u_trajectory_list', 'log_trajectory_list'])


In [3]:
opt_action_pos = cma_es['opt_action_pos']
opt_log = cma_es['opt_log']
print(opt_action_pos.shape)
print(opt_action_pos)
print(opt_log.keys())

torch.Size([5, 2])
tensor([[-0.8000,  0.8000],
        [ 0.6759, -0.3755],
        [-0.8000, -0.4952],
        [ 0.4774,  0.8000],
        [ 0.3750, -0.8000]], device='cuda:0')
dict_keys(['total_loss_trajectory', 'position_trajectory', 'us_trajectory', 'lower_level_log_trajectory', 'best_idx', 'best_loss', 'best_position', 'best_us'])


In [4]:
for key in opt_log.keys():
    print(key)
    if key == 'lower_level_log_trajectory':
        print(len(opt_log[key]))
    elif key == 'best_idx' or key == 'best_loss':
        print(opt_log[key])
    else:
        print(opt_log[key].shape)

total_loss_trajectory
(100, 10)
position_trajectory
(100, 10, 5, 2)
us_trajectory
(100, 10, 160, 24, 1)
lower_level_log_trajectory
100
best_idx
99
best_loss
0.7135311539750546
best_position
torch.Size([5, 2])
best_us
(160, 24, 1)


In [5]:
bilevel_path = 'design_opt_experiment/bilevel/3_5/design_opt_experiment_result_ICGAT_1.0_0.pkl'
bilevel = pickle.load(open(bilevel_path, 'rb'))
optimal_bilevel_path = 'design_opt_experiment/bilevel/optimal/3_5/optimal_experiment_result_ICGAT_1.0_0.pkl'
optimal_bilevel = pickle.load(open(optimal_bilevel_path, 'rb'))
print(bilevel.keys())
print(optimal_bilevel.keys())

dict_keys(['prev_graph', 'optimized_graph', 'design_opt_log'])
dict_keys(['x_trajectory_list', 'u_trajectory_list', 'log_trajectory_list'])


In [6]:
print(bilevel['optimized_graph'].nodes['action'].data['pos'].shape)
print(bilevel['optimized_graph'].nodes['action'].data['pos'])
print(bilevel['design_opt_log'].keys())

torch.Size([5, 2])
tensor([[-0.4253, -0.7466],
        [ 0.6062, -0.6960],
        [ 0.3234,  0.7707],
        [-0.7543, -0.3374],
        [ 0.7763,  0.4259]])
dict_keys(['total_loss_trajectory', 'position_trajectory', 'us_trajectory', 'lower_level_log_trajectory', 'best_idx', 'best_loss', 'best_position', 'best_us'])


In [7]:
for key in bilevel['design_opt_log'].keys():
    print(key)
    if key == 'lower_level_log_trajectory':
        print(len(bilevel['design_opt_log'][key]))
    elif key == 'best_idx' or key == 'best_loss':
        print(bilevel['design_opt_log'][key])
    else:
        print(bilevel['design_opt_log'][key].shape)

total_loss_trajectory
(101,)
position_trajectory
(101, 5, 2)
us_trajectory
(101, 160, 24, 1)
lower_level_log_trajectory
101
best_idx
97
best_loss
1.1030357480049133
best_position
(5, 2)
best_us
(160, 24, 1)


In [31]:
solver_names = ['bilevel']
new_solver_names = ['implicit']
num_x_list = [3, 4, 5]
num_heaters_list = [5, 10, 15, 20]
model_names = ['ICGAT', 'GAT']
num_repeats = 10

for (solver_name, new_solver_name) in zip(solver_names, new_solver_names):
    for model_name in model_names:
        if not os.path.exists('bilevel_opt_result/{}_{}'.format(new_solver_name, model_name)):
            os.makedirs('bilevel_opt_result/{}_{}'.format(new_solver_name, model_name))
        if not os.path.exists('bilevel_opt_result/optimal/{}_{}'.format(new_solver_name, model_name)):
            os.makedirs('bilevel_opt_result/optimal/{}_{}'.format(new_solver_name, model_name))
        for num_x in num_x_list:
            for num_heaters in num_heaters_list:
                design_opt_log = {}
                design_opt_log['total_loss_trajectory'] = []
                design_opt_log['position_trajectory'] = []
                design_opt_log['us_trajectory'] = []
                design_opt_log['lower_level_log_trajectory'] = []
                design_opt_log['best_idx'] = None
                design_opt_log['best_loss'] = float('inf')
                design_opt_log['best_position'] = []
                design_opt_log['best_us'] = []
                
                for idx in range(num_repeats):
                    design_opt_experiment_path = 'design_opt_experiment/{}/{}_{}/design_opt_experiment_result_{}_1.0_{}.pkl'.format(solver_name, num_x, num_heaters, model_name, idx)
                    design_opt_experiment = pickle.load(open(design_opt_experiment_path, 'rb'))['design_opt_log']                    
                    design_opt_log['total_loss_trajectory'].append(design_opt_experiment['total_loss_trajectory'])
                    design_opt_log['position_trajectory'].append(design_opt_experiment['position_trajectory'])
                    design_opt_log['us_trajectory'].append(design_opt_experiment['us_trajectory'])
                    design_opt_log['lower_level_log_trajectory'].append(design_opt_experiment['lower_level_log_trajectory'])
                    if design_opt_experiment['best_loss'] < design_opt_log['best_loss']:
                        design_opt_log['best_loss'] = design_opt_experiment['best_loss']
                        design_opt_log['best_idx'] = np.array([idx, design_opt_experiment['best_idx']])
                    design_opt_log['best_position'].append(design_opt_experiment['best_position'])
                    design_opt_log['best_us'].append(design_opt_experiment['best_us'])
                    
                design_opt_log['total_loss_trajectory'] = np.stack(design_opt_log['total_loss_trajectory'], axis=1)
                design_opt_log['position_trajectory'] = np.stack(design_opt_log['position_trajectory'], axis=1)
                design_opt_log['us_trajectory'] = np.stack(design_opt_log['us_trajectory'], axis=1)
                design_opt_log['best_position'].append(design_opt_experiment['best_position'])
                design_opt_log['best_us'].append(design_opt_experiment['best_us'])
                
                design_opt_experiment_path = 'design_opt_experiment/{}/{}_{}/design_opt_experiment_result_{}_1.0_{}.pkl'.format(solver_name, num_x, num_heaters, model_name, design_opt_log['best_idx'][0])
                design_opt_experiment = pickle.load(open(design_opt_experiment_path, 'rb'))
                
                opt_action_pos = design_opt_experiment['optimized_graph'].nodes['action'].data['pos']
                opt_result_log = {
                    'opt_action_pos': opt_action_pos,
                    'design_opt_log': design_opt_log
                }
                saved_path = 'bilevel_opt_result/{}_{}/{}_{}.pkl'.format(new_solver_name, model_name, num_x, num_heaters)
                pickle.dump(opt_result_log, open(saved_path, 'wb'))
                
                optimal_log_path = 'design_opt_experiment/{}/optimal/{}_{}/optimal_experiment_result_{}_1.0_{}.pkl'.format(solver_name, num_x, num_heaters, model_name, design_opt_log['best_idx'][0])
                optimal_log = pickle.load(open(optimal_log_path, 'rb'))
                
                saved_path = 'bilevel_opt_result/optimal/{}_{}/optimal_result_{}_{}.pkl'.format(new_solver_name, model_name, num_x, num_heaters)
                pickle.dump(optimal_log, open(saved_path, 'wb'))
                
                config_path = 'design_opt_experiment/{}/optimal/{}_{}/mpc_config_{}_1.0_{}.pkl'.format(solver_name, num_x, num_heaters, model_name, design_opt_log['best_idx'][0])
                config = pickle.load(open(config_path, 'rb'))
                
                saved_path = 'bilevel_opt_result/optimal/{}_{}/mpc_config_{}_{}.pkl'.format(new_solver_name, model_name, num_x, num_heaters)
                pickle.dump(config, open(saved_path, 'wb'))

In [32]:
solver_names = ['direct']
new_solver_names = ['single_layer']

for (solver_name, new_solver_name) in zip(solver_names, new_solver_names):
    for model_name in model_names:
        if not os.path.exists('bilevel_opt_result/{}_{}'.format(new_solver_name, model_name)):
            os.makedirs('bilevel_opt_result/{}_{}'.format(new_solver_name, model_name))
        if not os.path.exists('bilevel_opt_result/optimal/{}_{}'.format(new_solver_name, model_name)):
            os.makedirs('bilevel_opt_result/optimal/{}_{}'.format(new_solver_name, model_name))
        for num_x in num_x_list:
            for num_heaters in num_heaters_list:            
                design_opt_log = {}
                design_opt_log['total_loss_trajectory'] = []
                design_opt_log['position_trajectory'] = []
                design_opt_log['us_trajectory'] = []
                design_opt_log['best_idx'] = None
                design_opt_log['best_loss'] = float('inf')
                design_opt_log['best_position'] = []
                design_opt_log['best_us'] = []
                
                for idx in range(num_repeats):
                    design_opt_experiment_path = 'design_opt_experiment/{}/{}_{}/design_opt_experiment_result_{}_1.0_{}.pkl'.format(solver_name, num_x, num_heaters, model_name, idx)
                    design_opt_experiment = pickle.load(open(design_opt_experiment_path, 'rb'))['design_opt_log']                    
                    design_opt_log['total_loss_trajectory'].append(design_opt_experiment['total_loss_trajectory'])
                    design_opt_log['position_trajectory'].append(design_opt_experiment['position_trajectory'])
                    design_opt_log['us_trajectory'].append(design_opt_experiment['us_trajectory'])
                    if design_opt_experiment['best_loss'] < design_opt_log['best_loss']:
                        design_opt_log['best_loss'] = design_opt_experiment['best_loss']
                        design_opt_log['best_idx'] = np.array([idx, design_opt_experiment['best_idx']])
                    design_opt_log['best_position'].append(design_opt_experiment['best_position'])
                    design_opt_log['best_us'].append(design_opt_experiment['best_us'])
                
                design_opt_log['total_loss_trajectory'] = np.stack(design_opt_log['total_loss_trajectory'], axis=1)
                design_opt_log['position_trajectory'] = np.stack(design_opt_log['position_trajectory'], axis=1)
                design_opt_log['us_trajectory'] = np.stack(design_opt_log['us_trajectory'], axis=1)
                design_opt_log['best_position'].append(design_opt_experiment['best_position'])
                design_opt_log['best_us'].append(design_opt_experiment['best_us'])
                
                design_opt_experiment_path = 'design_opt_experiment/{}/{}_{}/design_opt_experiment_result_{}_1.0_{}.pkl'.format(solver_name, num_x, num_heaters, model_name, design_opt_log['best_idx'][0])
                design_opt_experiment = pickle.load(open(design_opt_experiment_path, 'rb'))
                
                opt_action_pos = design_opt_experiment['optimized_graph'].nodes['action'].data['pos']
                opt_result_log = {
                    'opt_action_pos': opt_action_pos,
                    'design_opt_log': design_opt_log
                }                
                saved_path = 'bilevel_opt_result/{}_{}/{}_{}.pkl'.format(new_solver_name, model_name, num_x, num_heaters)
                pickle.dump(opt_result_log, open(saved_path, 'wb'))
                
                optimal_log_path = 'design_opt_experiment/{}/optimal/{}_{}/optimal_experiment_result_{}_1.0.pkl'.format(solver_name, num_x, num_heaters, model_name)
                optimal_log = pickle.load(open(optimal_log_path, 'rb'))
                
                saved_path = 'bilevel_opt_result/optimal/{}_{}/optimal_result_{}_{}.pkl'.format(new_solver_name, model_name, num_x, num_heaters)
                pickle.dump(optimal_log, open(saved_path, 'wb'))
                
                config_path = 'design_opt_experiment/{}/optimal/{}_{}/mpc_config_{}_1.0.pkl'.format(solver_name, num_x, num_heaters, model_name)
                config = pickle.load(open(config_path, 'rb'))
                
                saved_path = 'bilevel_opt_result/optimal/{}_{}/mpc_config_{}_{}.pkl'.format(new_solver_name, model_name, num_x, num_heaters)
                pickle.dump(config, open(saved_path, 'wb'))

In [33]:
import pickle
import torch

solver_name = 'cma_es'

num_x_list = [3, 4, 5]
num_heaters_list = [5, 10, 15, 20]
for num_x in num_x_list:
    for num_heaters in num_heaters_list:
        path = 'bilevel_opt_result/{}/{}_{}.pkl'.format(solver_name, num_x, num_heaters)
        bilevel_opt_result = pickle.load(open(path, 'rb'))
        # bilevel_opt_result['opt_action_pos'] = bilevel_opt_result['opt_action_pos'].cpu().detach().numpy()
        bilevel_opt_result['opt_log']['best_position'] = bilevel_opt_result['opt_log']['best_position'].cpu().detach().numpy()
        pickle.dump(bilevel_opt_result, open(path, 'wb'))

RuntimeError: Attempting to deserialize object on CUDA device 1 but torch.cuda.device_count() is 1. Please use torch.load with map_location to map your storages to an existing device.

In [37]:
num_x = 3
num_heaters = 5
path = 'bilevel_opt_result/{}/{}_{}.pkl'.format(solver_name, num_x, num_heaters)
bilevel_opt_result = pickle.load(open(path, 'rb'))

print(bilevel_opt_result.keys())
print(type(bilevel_opt_result['opt_action_pos']))
opt_log = bilevel_opt_result['opt_log']

dict_keys(['opt_action_pos', 'opt_log'])
<class 'numpy.ndarray'>


In [39]:
for key in opt_log.keys():
    print(key)
    print(type(opt_log[key]))

total_loss_trajectory
<class 'numpy.ndarray'>
position_trajectory
<class 'numpy.ndarray'>
us_trajectory
<class 'numpy.ndarray'>
lower_level_log_trajectory
<class 'list'>
best_idx
<class 'int'>
best_loss
<class 'numpy.float64'>
best_position
<class 'torch.Tensor'>
best_us
<class 'numpy.ndarray'>


In [40]:
print(opt_log['best_position'].device)

cuda:0


In [46]:
lower_level_log = opt_log['lower_level_log_trajectory'][0][0]

In [49]:
for key in lower_level_log.keys():
    print(key)
    print(type(lower_level_log[key]))

target_xs
<class 'numpy.ndarray'>
individual_loss
<class 'numpy.ndarray'>
total_loss
<class 'numpy.ndarray'>
time
<class 'numpy.ndarray'>
total_runtime
<class 'float'>
best_idx
<class 'numpy.ndarray'>
best_us
<class 'numpy.ndarray'>
best_loss
<class 'numpy.float64'>
